In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime 
import sys
%matplotlib inline
evaluation=pd.read_csv('submit.txt')
product_info=pd.read_csv('product_info.txt')
product_quantity=pd.read_csv('product_quantity.txt')
Mytrain_day=pd.read_csv('Mytrain_day.csv')

In [6]:
def random_read_train_data(k,table):    
    _Mytrain_day=table.take(np.random.permutation(len(table))[:k])   
    _features = _Mytrain_day.drop('ciiquantity',axis=1)
    _targets = _Mytrain_day['ciiquantity']
    _Myfeatures=_features.as_matrix()
    _Mytargets=_targets.as_matrix()
    _Mytargets.shape[0]
    _Mytargets.shape = (_Mytargets.shape[0], 1)
    _Mytargets.transpose()
    return _Myfeatures,_Mytargets

In [11]:
def add_layer(inputs, in_size, out_size, activation_function=None):    
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
in_shape=13
epochs=1000
batch=50000
l_r=0.01
x = tf.placeholder(tf.float32, shape=[None, in_shape])
y = tf.placeholder(tf.float32, [None,1])
l1 = add_layer(x, in_shape, 10, activation_function=tf.nn.sigmoid)
prediction = add_layer(l1, 10, 1, activation_function=None)

loss = tf.reduce_mean(tf.reduce_sum(tf.square(y - prediction),
                     reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(l_r).minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())    
    # Training cycle
for epoch in range(epochs):
    start = datetime.now()
    #tf_features,tf_targets = cut_train_data(epoch*1000,(epoch+1)*1000)
    tf_features,tf_targets = random_read_train_data(batch,Mytrain_day)
    tf_features_text,tf_targets_text = random_read_train_data(batch,Mytrain_day)
    sess.run(train_step, feed_dict={x: tf_features, y: tf_targets})
    temploss = sess.run(loss, feed_dict={x: tf_features_text, y: tf_targets_text})
    end = datetime.now()
    sys.stdout.write('\rEpoch {:>2} Loss: {:.4f} time: {}'.format(epoch + 1,temploss,end-start))

Epoch 1000 Loss: 53.5624 time: 0:00:00.189104

In [12]:
def getweek(product_date):
    yyyy=int(product_date[0:4])
    mm=int(product_date[5:7])
    dd=int(product_date[8:10])
    return datetime(yyyy,mm,dd).weekday()+1

def addweekday(table):
    table['week']=table['product_date'].apply(getweek)
    dummies = pd.get_dummies(table['week'], prefix='week', drop_first=False)
    table = pd.concat([table, dummies], axis=1)
    table = table.drop('week', axis=1)
    table = table.drop('product_date', axis=1)
    return table

In [ ]:
import calendar
def get_evaluation(x):
    _date=evaluation[evaluation.index==x].product_month.as_matrix()[0]
    _id=evaluation[evaluation.index==x].product_id.as_matrix()[0]
    _yyyy=int(_date[0:4])
    _mm=int(_date[5:7])
    _days=calendar.monthrange(_yyyy,_mm)[1]
    _table=pd.DataFrame({'product_date':range(_days)})
    _table['product_date']=_table['product_date'].apply(lambda x:_date[0:8]+str(x+1).zfill(2))
    _table['product_id']=_id
    _features=['eval','eval2','eval3','eval4','voters','maxstock']
    for feature in _features:
        value=product_info[product_info.product_id==_id][feature].as_matrix()[0]
        mean, std = product_info[feature].mean(), product_info[feature].std()
        value = (value - mean)/std
        _table[feature]=value
    _table=addweekday(_table)
    _table=_table.drop('product_id', axis=1)
    #_table=addfeature(_table,_features)
    #_table=Scalingfeature(_table,_features)
    _table=_table.as_matrix()
    return _table
y_array=np.sum(sess.run(prediction, feed_dict={x: get_evaluation(0)}))
print(y_array)
ciiquantity_month=np.zeros(evaluation.shape[0])
#'''
start = datetime.now()
for i in range(evaluation.shape[0]):
#for i in range(1000):
    _date=evaluation[evaluation.index==i].product_month.as_matrix()[0]
    _mm=int(_date[5:7])
    
    month_data=get_evaluation(i)
    month_data_sum=np.sum(sess.run(prediction, feed_dict={x: month_data}))
    if(_mm==10):
        print(_mm)
        ciiquantity_month[i]=month_data_sum*2
    else:
        ciiquantity_month[i]=month_data_sum*1.2
    if i % 100 == 0:
        end = datetime.now()        
        sys.stdout.write('\ri:{}/{} ciiquantity month:{} time:{}'.format(i+1,evaluation.shape[0],month_data_sum,end-start))
        start = datetime.now()
#'''
evaluation['ciiquantity_month']=ciiquantity_month
evaluation.to_csv('my_ansower_quick_nn.csv',index=False)
evaluation

158.54
i:31901/56000 ciiquantity month:179.87612915039062 time:0:00:01.886002